# Lab session 01 - Introduction to Machine Learning in Python
by Dr Ivan Olier


## Introduction
For the following questions you should review your knowledge of Python. You can follow this link to an extensive tutorial on how to program with Python: https://python.swaroopch.com/
In addition, you can find on Teams a Python tutorial as a Juptyer Notebook. You can open this tutorial on Google Colab (colab.research.google.com) and follow it step-by-step. 

## Exercise 1

- Implement a Python function that estimates the coefficients β of a multivariate linear regression model.
- Use the implemented function to estimate the parameters (β) for the following training dataset:

|row|X1|X2|y|
|-----|-------|------|----|
|1|0.1|2.2|5.4|
|2|0.9|3.9|4.1|
|3|2.1|6.1|2.9|
|4|2.7|7.7|1.1|
|5|4.2|10.3|0.3|

- Implement a small code in Python that estimates the performance metrics studied in the lecture (i.e., RSS, RSE, R2, MSE, and RMSE) of the above multiple linear regression model.

### Solution


In [ ]:
import numpy as np

In [ ]:
# implements part a) -> function to estimate MLR coeffcients (betas): beta = (Xt*X)^-1*Xt*y
from numpy.linalg import inv  # inverse matrix function
def mlr_coeff(X,y):  
    X = np.c_[np.ones(len(X)), X]  # append 1s column (beta0)
    return inv((X.T) @ X) @ X.T @ y

In [ ]:
Xtrain = np.array([[0.1, 2.2],
                   [0.9, 3.9],
                   [2.1, 6.1],
                   [2.7, 7.7],
                   [4.2, 10.3]])
Xtrain

In [ ]:
ytrain = np.array([[5.4, 4.1, 2.9, 1.1, 0.3]]).T
ytrain

In [ ]:
Beta = mlr_coeff(Xtrain,ytrain)
Beta

In [ ]:
def mlr_predict(BETA, Xnew):
    Xnew = np.c_[np.ones(len(Xnew)), Xnew]
    return Xnew @ BETA

In [ ]:
y_pred = mlr_predict(BETA=Beta, Xnew=Xtrain)
y_pred

In [ ]:
def mlr_scores(y_true, y_pred):
    err = y_true - y_pred
    RSS = ( err.T @  err ).item(0) # RSS is still an array. converts to scalar with `item`
    n = len(y_true)
    RSE = np.sqrt(RSS/(n-2))
    ybar = np.mean(y_true)
    R2 = ( 1 - RSS / ( (y_true - ybar).T @ (y_true - ybar) ) ).item(0)
    MSE = RSS/n
    RMSE = np.sqrt(MSE)
    return RSS, RSE, R2, MSE, RMSE

In [ ]:
RSS, RSE, R2, MSE, RMSE = mlr_scores(y_true=ytrain, y_pred=y_pred)
print("RSS = ", RSS)
print("RSE = ", RSE)
print("R2 = ", R2)
print("RMSE = ", RMSE)



## Exercise 2
Let assume a logistic regression model for the gestational diabetes (see lecture slides) with coefficients:

|Coefficient|Value|
|----|---|
|(Intercept)|-10.96|
|Pregnancies|0.18|
|Glucose|0.04|
|BloodPressure|0.005|
|SkinThickness|-0.008|
|Insulin|0.002|
|BMI|0.118|
|DiabetesPedigreeFunction|1.71|
|Age|-0.004|

- Write a piece of code in Python that implements a function that takes in an individual test data observation (X) and outputs its predicted probability (P(X)). 
- Use your implemented function to predict the risk of developing diabetes. You can test a few observations taken from the lecture slides. 

### Solution

In [ ]:
def pred_prob(X):
  betas = np.array([0.18, 0.04, 0.005, -0.008, 0.002, 0.118, 1.71, -0.004])
  bx = -10.96 + X @ betas
  return(np.exp(bx)/(1+np.exp(bx)))

In [ ]:

X = np.array([[6, 148, 72, 35, 0, 33.6, 0.627, 50],
              [1, 85, 66, 29, 0, 26.6, 0.351, 31],
              [8, 183, 64, 0, 0, 23.3, 0.672, 32]])

pred_prob(X)


## Exercise 3
We will use logistic regression to produce a classification model for *gestational diabetes*. The classification task will be to predict the condition in pregnant women, or in other words, to classify pregnant women suffering the condition from healthy women. The *pima indians diabetes database* will be used, which is publicly availble. It is also available under the following link:
https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/diabetes.csv

- Load the “diabetes.csv” into a Python’s Pandas data frame
- Report # of variables, # of observations, and # of missing values
- Explore all the variables by producing histograms and detect possible outliers, suspicious values, etc.
- Drop any row/column with suspicious values
- Split the data into training and test subsets
- Train a logistic regression model using the training subset
- Predict the risk of developing diabetes using the test subset.
- Compare the predicted probabilities with the ones estimated in Exercise 2.
- Plot the ROC and PR curves.
- Estimate AUROC and AUPR values. 
- Explain the quality of the model performance.

### Solution


In [ ]:
# Import useful libraries 
import pandas as pd     # to handle data frames
import numpy as np      # to manipulate matrices/vectors
import matplotlib.pyplot as plt     # to generate plots

# to allow for plots in notebooks
%matplotlib inline  

In [ ]:
# "diabetes.csv": following below link to the dataset
dset = pd.read_csv("https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/diabetes.csv")  # reads csv files
dset.head() # shows the top rows of data

In [ ]:
# Option 2
#from google.colab import files
#uploaded = files.upload()
#import io
#dset = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))
#dset.head()

In [ ]:
dset.shape    # returns data dimensionality

In [ ]:
dset.describe()   # shows some basic statistics

In [ ]:
plot1 = dset.hist(figsize=[10,10])    # displays histograms of the DataFrame columns

* are there any issues with the *diabetes* dataset?

In [ ]:
# Use `replace` to replace values in the columns. Here we replace any "0" by "NaN" (not a number - which represents missing values in Python)
dset = dset.replace({'BMI' : 0, 'BloodPressure' : 0, 'Glucose' : 0, 'Insulin' : 0, 'SkinThickness' : 0}, np.nan)
dset.describe()

In [ ]:
dset.isna().sum()     # shows the number of missings per column

In [ ]:
dset = dset.dropna()    # drops rows with missing values
dset.describe()

* Now we could say the data is ready for use.

### Using the `sklearn` library
*scikit-learn* library or "sklearn", in short, is the most popular library for machine learning modelling in Python. 
Please, follow this link for it official documentation: https://scikit-learn.org/stable/

We will use the library for most of the *Data Mining* module. In this tutorial, it will be used for modelling the *diabetes* data with Logistic Regression and for model evaluation.

In the following steps, we will:

1. split the data into a traning and a test subset.
2. train a logistic regression model using the training subset.
3. predict new outputs (classes and class probabilities) using the test subset.
4. evaluate the model performance


In [ ]:
from sklearn.model_selection import train_test_split      # imports function to splits the data
X_train, X_test, y_train, y_test = train_test_split(dset.drop('Outcome', axis=1), dset['Outcome'], test_size=0.3, random_state=123)  # splits 70% training, 30% test

In [ ]:
from sklearn.linear_model import LogisticRegression   # imports the logistic regression module implemented in sklearn

In [ ]:
mdl = LogisticRegression(max_iter = 1000)    # sets up the algorithm (or learner)

In [ ]:
mdl.fit(X_train, y_train)       # trains the model using the training set

In [ ]:
mdl.coef_       # shows the learnt coefficients (or betas)

In [ ]:
y_class = mdl.predict(X_test)     # predicts new classes using the test set 

In [ ]:
y_class

In [ ]:
y_prob = mdl.predict_proba(X_test)    # predicts class probabilities 
y_prob[0:10,]     # the output is a 2-col array: one for each class

**Note: the below code might not work if you have an early version of *scikit-learn*. If fails, please have a look at the code in comments.**

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(y_test, y_prob[:,1])

`RocCurveDisplay.from_predictions` takes the true labels (`y_test`) and the predicted probabilities of the class 1. Note that `y_prob` is a 2D matrix. The first column (i.e. column 0) contains the probabilities of being the class 0, whilst the second column (i.e. column 1), the class 1, which is the one of interest.

In [ ]:
## If above code fails due to version compatibilities, then you can try the following code:

# from sklearn.metrics import roc_curve, auc
# fpr, tpr, thres = roc_curve(y_test, y_prob[:,1])
# auc_score = auc(fpr, tpr)
# plt.figure()
# plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % auc_score)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
PrecisionRecallDisplay.from_predictions(y_test, y_prob[:,1])

In [ ]:
# computes TPR, FPR, and thresholds
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob[:,1])


In [ ]:
optim_th = thresholds[np.argmax(tpr-fpr)] # takes the one with maximal sum of true positive and true negative rates (1-fpr)
optim_th

In [ ]:
y_class_opt = np.where(y_prob[:,1] > optim_th, 1, 0)
y_class_opt

In [ ]:
#Accuracy:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_class_opt)

In [ ]:
# Confusion matrix:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_true=y_test, y_pred=y_class_opt)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mdl.classes_).plot()

## Exercise 4
Repeat a similar analysis, but now using the “SAheart” dataset instead. The dataset is available on:
https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/SAheart.csv 

The South African Heart dataset (SAHeart): The dataset is publicly available, just type its name in Google. The dataset is a retrospective sample of males in a heart-disease high-risk region of the Western Cape, South Africa. There are roughly two controls per case of Coronary Heart Disease (CHD). Many of the CHD positive men have undergone blood pressure reduction treatment and other programs to reduce their risk factors after their CHD event. In some cases the measurements were made after these treatments. These data are taken from a larger dataset, described in Rousseauw et al, 1983, South African Medical Journal.
This is the set of variables in the dataset:

- sbp - systolic blood pressure
- tobacco - cumulative tobacco (kg)
- ldl - low densiity lipoprotein cholesterol
- adiposity
- famhist - family history of heart disease (Present, Absent)
- typea - type-A behavior
- obesity
- alcohol - current alcohol consumption
- age - age at onset
- chd - response, coronary heart disease

The aim is to predict the risk of CHD as a function of the other variables. This is essentially a classification task with two classes: CHD/No CHD (coded as 1 and 0, respectively).



In [ ]:
# Your answer here